Proyecto Integrador – Fleetlogix

Este notebook documenta la ejecución de queries básicas, intermedias y complejas sobre la base de datos PostgreSQL de Fleetlogix. Cada consulta incluye:

Explicación de qué hace la query

Problema de negocio que resuelve

Código SQL

Resultados esperados / interpretación

QUERIES BÁSICAS
Query 1 – Contar vehículos por tipo
Qué hace: Agrupa todos los vehículos por vehicle_type y cuenta cuántos hay en cada categoría. Problema de negocio: Permite evaluar la capacidad operativa y detectar si algún tipo de vehículo está sobre o subrepresentado.

In [ ]:
SELECT vehicle_type, COUNT(*) AS total
FROM vehicles
GROUP BY vehicle_type
ORDER BY total DESC;

Query 2 – Conductores con licencia a vencer en 30 días
Qué hace: Filtra conductores cuya fecha de vencimiento ocurre dentro de los próximos 30 días. Problema de negocio: Identifica riesgo operacional por licencias próximas a vencer.

In [ ]:
SELECT driver_id, first_name, last_name, license_expiry
FROM drivers
WHERE license_expiry BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '30 days';

Query 3 – Total de viajes por estado
Qué hace: Agrupa todos los viajes por su estado y devuelve un conteo. Problema de negocio: Permite monitorear el flujo de operaciones y detectar congestión o retrasos.

In [ ]:
SELECT status, COUNT(*) AS total
FROM trips
GROUP BY status;

QUERIES INTERMEDIAS
Query 4 – Total de entregas por ciudad (últimos 60 días)
Qué hace: Une deliveries → trips → routes para obtener la ciudad destino y cuenta entregas. Problema de negocio: Identifica ciudades con mayor volumen de entregas para asignar recursos.

In [ ]:
SELECT r.destination_city, COUNT(d.delivery_id) AS total_entregas,
       SUM(d.package_weight_kg) AS peso_total
FROM deliveries d
JOIN trips t ON d.trip_id = t.trip_id
JOIN routes r ON t.route_id = r.route_id
WHERE d.scheduled_datetime >= CURRENT_DATE - INTERVAL '60 days'
GROUP BY r.destination_city
ORDER BY total_entregas DESC;

Query 5 – Conductores activos y carga de trabajo
Qué hace: Filtra conductores activos y cuenta viajes realizados. Problema de negocio: Ayuda a balancear la carga laboral y prevenir burnout.

In [ ]:
SELECT d.driver_id, d.first_name, d.last_name, COUNT(t.trip_id) AS total_viajes
FROM drivers d
JOIN trips t ON d.driver_id = t.driver_id
WHERE d.status = 'active'
GROUP BY d.driver_id, d.first_name, d.last_name
ORDER BY total_viajes DESC;

Query 6 – Promedio de entregas por conductor (últimos 6 meses)
Qué hace: Calcula total y promedio mensual de entregas por conductor. Problema de negocio: Mide productividad reciente y detecta estacionalidad.

In [ ]:
WITH entregas AS (
    SELECT d.driver_id, COUNT(del.delivery_id) AS total_entregas
    FROM deliveries del
    JOIN trips t ON del.trip_id = t.trip_id
    JOIN drivers d ON t.driver_id = d.driver_id
    WHERE del.scheduled_datetime >= CURRENT_DATE - INTERVAL '6 months'
    GROUP BY d.driver_id
)
SELECT driver_id, total_entregas, ROUND(total_entregas/6.0,2) AS promedio_mensual
FROM entregas
ORDER BY promedio_mensual DESC;

QUERIES COMPLEJAS
Query 9 – Costo de mantenimiento por kilómetro (CTE)
Qué hace: Calcula costo por km usando CTEs. Problema de negocio: Determina costo operativo real por vehículo.

In [ ]:
WITH km AS (
    SELECT vehicle_id, SUM(r.distance_km) AS total_km
    FROM trips t
    JOIN routes r ON t.route_id = r.route_id
    GROUP BY vehicle_id
),
costos AS (
    SELECT vehicle_id, SUM(m.cost) AS total_cost
    FROM maintenance m
    GROUP BY vehicle_id
)
SELECT k.vehicle_id, k.total_km, c.total_cost,
       ROUND(c.total_cost / NULLIF(k.total_km,0),2) AS costo_por_km
FROM km k
JOIN costos c ON k.vehicle_id = c.vehicle_id;

Query 10 – Ranking de conductores por eficiencia
Qué hace: Usa RANK() para ordenar conductores por cantidad de viajes. Problema de negocio: Genera ranking objetivo de rendimiento.

In [ ]:
SELECT driver_id, COUNT(trip_id) AS total_viajes,
       RANK() OVER (ORDER BY COUNT(trip_id) DESC) AS ranking
FROM trips
GROUP BY driver_id
ORDER BY ranking;
